In [ ]:
from book_funs3 import *



# Reproducing-kernel methods for machine learning

## Purpose of this chapter

We begin with the description ofreproducing-kernel methods which are adapted to problems arising in the theory of machine learning. We discuss two of the main ingredients required in the design of our CodPy algorithms. 

* First of all, our algorithms depend on the choice of a reproducing-kernel space and on the use of transformation maps which we apply to collection of basic kernels in order to adapt them to any particular problem.

* Second, we also define the kernel-based notions of (mesh-free) discrete differential operators which are relevant for machine learning.

The notions presented here provide us with the key building blocks for our basic algorithms of machine learning, but also to deal with problems involving partial differential operators. In the following chapters of this monograph we will next presentmore advanced algorithms and various applications. 

For a description of the framework of interest we need some notation. A set of $N_x$ variables in $D$ dimensions is given, denoted by $X \in \mathbb{R}^{N_x\times D}$, together with a $D_f$-dimensional vector-valued data function $f(X) \in \mathbb{R}^{N_x \times D_f}$ which represents the *training values* associated with the *training set* $X$. The input data therefore consists of 
$$
(X,f(X)) := \{x^n, f(x^n)\}_{n = 1,\dots,N_x}, \qquad X \in \mathbb{R}^{N_x \times D}, \qquad f(X) \in \mathbb{R}^{N_x \times D_f}.
$$
We are interested in predicting the so-called *test values* $f_Z \in \mathbb{R}^{N_z \times D_f}$ on a new set of variables called the *test set* $Z \in \mathbb{R}^{N_z \times D}$: 
\begin{equation} 
(Z,f_Z) := \{z^n, f_z^n\}_{n = 1,\dots,N_z}, \qquad Z \in \mathbb{R}^{N_z \times D}, \qquad f_Z \in \mathbb{R}^{N_z \times D_f}. (\#eq:PP)
\end{equation}
Note in passing that all of the examples and numerical experiments given below will be based on the following choice of function consisting of the sum of a periodic function and an increasing function (in each direction):
\begin{equation}
f(x) = f(x_1, \ldots, x_D) = \prod_{d=1,\ldots,D} \cos (4\pi x_d) + \sum_{d=1,\ldots,D} x_d, \qquad x \in \mathbb{R}^D. 
\end{equation}


In [ ]:
knitr::kable(cbind(py$D,py$Nx,py$Ny,py$Nz), col.names = c('$D$', '$N_x$', '$N_y$', '$N_z$'), caption = " choice of dimension for data extrapolation", escape = FALSE) %>%
  kable_styling(latex_options = "HOLD_position")


At this stage, we do not explain all of our notation but provide some numerical illustration only, concerning the prediction $(Z,f_Z)$ from the training set $(X,f(X))$. In fact, in addition to our notation $X \in \mathbb{R}^{N_x\times D}$ and $Z \in \mathbb{R}^{N_z\times D}$ together with $f(X) \in \mathbb{R}^{N_x\times D_f}$ and $f(Z) \in \mathbb{R}^{N_z\times D_f}$, we also introduce an extra variable denoted by $Y$ below and we distinguish between several cases. The choice $N_x = N_z$ will correspond to a *data extrapolation*, as will be explained in Section \@ref(extrapolation-interpolation-and-projection) below. On the other hand, a choice $N_y<<N_x$ will correspond to a *data projection*, as will be also explained below.   



In [ ]:
dummy,y1,dummy,dummy,fy1,dummy,dummy,dummy,dummy,dummy,dummy =  data_random_generator_.get_raw_data(D=D,Nx=Nx,Ny=Ny/100,Nz=Nz)



In [ ]:
knitr::kable(cbind(py$D,py$Nx,length(py$y1),py$Nz), col.names = c('$D$', '$N_x$', '$N_y$', '$N_z$'), caption = "A choice of dimensions for data projection", escape = FALSE) %>%
  kable_styling(latex_options = "HOLD_position")


In [ ]:
dummy,y2,dummy,dummy,fy2,dummy,dummy,dummy,dummy,dummy,dummy =  data_random_generator_.get_raw_data(D=D,Nx=Nx,Ny=Ny/100,Nz=Nz)



Hence, Figure \@ref(fig:xfxyfyzfz) provides us with a typical illustration of results of machine learning, and we will particularly on the choice made in the first test throughout the following discussion. The left-hand plots show the (variables, value) training set $(X, f(X))$, while the right-hand plot displays the (variables, values) test set $(Z, f(Z))$. The plots in the middle display the (variables, values) parameter set $(Y ,f(Y))$, whose importance will be explained later on: in short, the choice of $Y$ closely determines not only the overall accuracy of the algorithm, but also its computational cost. Having provided a broad illustration we can now proceed and define all relevant concept in full details. 



In [ ]:
multi_plot([(x,fx),(y,fy),(z,fz),(x,fx),(y2,fy2),(z,fz),(x,fx),(z,fz),(z,fz)],plot_trisurf,mp_ncols = 3, projection='3d',mp_max_items = -1, f_names=["training set","parameter set (extrapolation)","test set","training set","parameter set (projection)","test set","training set","parameter set (interpolation)","test set"])



## Fundamental notions for supervised learning 

### Preliminaries 

**Positive kernels and kernel matrices**. Let $k=k(x,y): \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}$ be a symmetric real-valued function, that is, a function satisfying $k(x, y)=k(y, x)$. Given two collection $X = (x^1, \cdots, x^{N_x})$ and $Y = (y^1, \cdots, y^{N_y})$  of points in $\mathbb{R}^D$, we define the associated *kernel matrix* $K(X,Y) := \big(k(x^n,y^m) \big) \in \mathbb{R}^{N_x \times N_y}$ by 
\begin{equation}
    K(X, Y) = 
    \left( 
    \begin{array}{ccc} 
        k(x^1,y^1) & \cdots & k(x^1,y^{N_y}) \\
        \ddots & \ddots & \ddots \\
         k(x^{N_x},y^1) & \cdots & k(x^{N_x},y^{N_y})
    \end{array}
    \right).
\end{equation}
We say that $k$ is a positive kernel if, for any collection of distinct points $X \in \mathbb{R}^{N_x \times D}$ and for any (non-identically vanishing) $c^1,...,c^{N_x} \in \mathbb{R}^{N_x}$,
\begin{equation}
\sum_{1 \leq i,j \leq  N_x} c^i c^j k(x^i,x^j) > 0. 
\end{equation}

When $N_x=N_y$, the square matrix $K(X,Y)$ is called a Gram matrix. The dimension of the null space of the matrix $K(X,Y)$ is usually $N_x \times N_y$, except typically for some kernels, as can be checked in the section on kernel engineering (Section \@ref(kernel-engineering)).

If $K(X,Y)$ is positive only on a certain sub-manifold of $\mathbb{R}^D$, we say that the kernel is *conditionally positive*, that is, the ositivity condition golds only if $X, Y$ belongs to this sub-manifold.

We always use positive or conditionally positive kernels. The available kernels in our library are listed in the table below. 

| | Kernel name | $k(x, y)    |
|---|---|---|
|1. | Dot product | $k(x, y) = x^Ty$   |
|2.  | RELU   | $k(x, y) =\max(x-y,0)$  |
|3.    | Gaussian  | $k(x, y) = \exp(-\pi|x - y|^2)$  |
|4.   | Periodic Gaussian   | $\sigma^2 \exp(-\frac{2}{l^2}\sin^2 ( \pi \frac{|x-y|}{p})$   |
|5.   | Matern norm  |   |
|6.   | Matern tensor  |   |
|7.   | Matern periodic  |   |
|8.   |  Multiquadric norm |   |
|9.   | Multiquadric tensor  |   |
|10.   | Sincard square tensor  |   |
|11.   | Sincard tensor   |   |
|12.   | Tensor norm  |   |
|13.   |  Truncated norm |   |
|14.   |  Truncated periodic |   |


Here, for the Gaussian kernel, $l$ denotes the length scale, $p$ the period, and $\sigma$ the mplitude. A scaling of the basic kernels may be required in order to handle input data, which is exactly the purpose of the maps, discussed below.

\begin{example}
Gaussian kernel reads (and is used by default in the CodPy library) 
\begin{equation} 
k(x, y) = \exp(-\pi|x - y|^2).
(\#eq:Gaussian)
\end{equation}
\end{example}

\begin{example} 
A mapping $S:\mathbb{R}^D \mapsto \mathbb{R}^P$ begin given, consider the family of kernels 
$$
k(x,y) = g(<S(x),S(y)>_{\mathbb{R}^P}), \qquad x,y \in \mathbb{R}^D,
$$
where $g$ is called an activation function. In particular, the scalar products between the collections of successive powers of $x_d$ and $y_d$, denoted by 
$$
k(x,y)=<(1,x,x^Tx,\ldots), (1,y,y^Ty,\ldots)>
$$
defines a kernel corresponding to a linear regression over a polynomial basis. It is positive, but the null space of the associated matrix kernel is non-empty. The so-called RELU kernel given by 
$$
k(x, y)=\max(<x,y>+c,0)
$$
($c$ being a constant) is a conditionally positive kernel
\end{example}


In [ ]:
Knm = op.Knm(x,y, **get_codpy_param_chap3())
#print("Knm shape:",Knm.shape)


Let us consider the kernel to be *tensornorm* (discussed below) and let us refer to Section \@ref(brief-overview-of-methods-of-machine-learning) for the description of the relevant parameters in our algorithm. We then display some typical values of the kernel matrix, in Table \@ref(tab:450),, which are computed using our function *op.Knm* in CodPy.



In [ ]:
knitr::kable(py$Knm[0:4,0:4], caption = "First four rows and columns of the kernel matrix $K(X,Y)$")%>%
kable_styling(latex_options = "HOLD_position")


**Inverse of a kernel matrix**. The inverse of a kernel matrix is denoted $K(X, Y)^{-1}$ and is computed, if we choose $X=Y$, as follows:
$$
  K(X, X)^{-1} = (K(X, X) + \epsilon I_d)^{-1}.
$$
When $X \neq Y$, this inverse is computed using a least-square approach, namely 
$$
  K(X,Y)^{-1} = (K(Y,X)K(X,Y)+ \epsilon I_d)^{-1}K(Y,X)
$$
The so-called Tikhonov parameter $\epsilon$ represents a regularization parameter (and, by default in CodPy, takes the value $\epsilon = 10^{-8}$). 


In [ ]:
Knm_inv =op.Knm_inv(x=x,y=y)
inv_shape = Knm_inv.shape



Table \@ref(tab:455) displays the first four rows and columns of the inverse of the kernel matrix $K(X,Y)^{-1} \in \mathbb{R}^{N_y\times N_x}$ when $N_x = N_y$. 


In [ ]:
knitr::kable(py$Knm_inv[0:4,0:4], caption = "First four rows and columns of an inverted kernel matrix $K(X,Y)^{-1}$")%>%
kable_styling(latex_options = "HOLD_position")


The matrix product $K(X, Y)K(X, Y)^{-1}$ in Table \@ref(tab:455) is just a projection operator. It might not be the identity depending on the setup of interest, for one of the following reasons: 

- If $N_x \neq N_y$.

- If the Tikhonov regularization parameter $\epsilon >0$ is non-zero. While the user can choose $\epsilon = 0$, some performance issues may arise. Namely, if the kernel is not (unconditionally) positive, then the CodPy library might raise an ``exception'', and will switch from the standard inversion of matrices to an adapted inversion (of non-invertible matrices) which can be more computationally costly.

- If the kernel under consideration is such that $K(X, X)K(X, X)^{-1}$ does not have a full rank, for instance if a linear regression kernel is used; see the section on kernel engineering (Section \@ref(kernel-engineering)). In which case this matrix is a projection over the null space of the matrix $K(X,X)$.

**Distance matrices**. The notion of distance matrix defined now is a simple and very convenient tool within the framework of kernel methods. To any positive kernel $k : \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}$ we can associated the distance function $d_k(x,y)$ for all $x \in \mathbb{R}^D$ and $y \in \mathbb{R}^D$, defined by 
\begin{equation}
d_k(x,y) = k(x,x) + k(y,y) - 2k(x,y). (\#eq:ES)
\end{equation}
For a positive kernel, this expression is continuous, non-negative, and satisfies $d(x,x) = 0$.

For any collections $X = (x^1,...,x^{N_x})$ and $Y = (y^1,...,y^{N_y})$ of points in $\mathbb{R}^D$ we define a the associated *distance matrix* $D(X,Y) \in \mathbb{R}^{N_x \times N_y}$ by 
\begin{equation}
    D(X,Y) = 
    \left( 
    \begin{array}{ccc} 
        d_k(x^1,y^1) & \cdots & d_k(x^1,y^M) \\
        \ddots & \ddots & \ddots \\
         d_k(x^N,y^1) & \cdots & d_k(x^N,y^M)
    \end{array}
    \right).
\end{equation}


In [ ]:
Dnmxy =op.Dnm(x=x,y=y)



Table \@ref(tab:485) displays the first four columns of the kernel-based distance distance matrix $D(X,Y)$, and obviously the diagonal of this matrix vanishes.



In [ ]:
knitr::kable(py$Dnmxy[0:4,0:4], caption = "First four rows and columns of a kernel-based distance matrix $D(X,Y)$") %>%
kable_styling(latex_options = "HOLD_position")


### Methodology of the CodPy algorithms

Our algorithms provide one with general functions in order to make predictions in \@ref(eq:PP) from the choice of a kernel. More precisely, the following operator (with $A^{-1} := (A^TA)^{-1}A^T$ denoting the least-square inverse)
\begin{equation}
 f_z := \mathcal{P}_{k}(X,Y,Z)f(X):= K(Z,Y)K(X,Y)^{-1}f(X),\quad K(Z,Y) \in \mathbb{R}^{N_z \times N_y}, K(X,Y) \in \mathbb{R}^{N_x \times N_y} (\#eq:P)
\end{equation}
defines a supervised learning machine which we call a **feed-forward** machine. We also consider $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_x}$ as a **projection operator** and it is well-defined once a kernel $k$ has been chosen. Observe that two factors arise in \@ref(eq:P), namely the so-called kernel matrix $K(X,Y)$ (discussed below) and the **projection set of variables** denoted by $Y \in \mathbb{R}^{N_y \times D}$.  To motivate the role of the later, let us consider two particular operators that do not depend upon $Y$: 
\begin{align}
 &\text{Extrapolation operator: } \mathcal{P}_{k}(X, Z) = K(Z,X)K(X, X)^{-1},
 \\
& \text{Interpolation operator: }\mathcal{P}_{k}(X, Z) = K(X, Z)^{-1}K(X,X). (\#eq:EI)
\end{align}
These operators sometimes generate computational issues, due to the fact that the kernel matrix $K(X, X) \in \mathbb{R}^{N_x \times N_x}$ must be inverted \@ref(eq:P) and this is a rather costly computational step in presence of a large set of input data. This is our motivation for introducing the additional variable $Y$ which has the effect to lower the computational cost. It reduces the overall algorithmic complexity of \@ref(eq:P) to the order of
$$
D \, \big( (N_y)^3 + (N_y)^2 N_x + (N_y)^2 N_z \big). 
$$
Most importantly, the projection operator $\mathcal{P}_{k}$ is \textit{linear} in term of, both, input and output data. Hence, while keeping the set $Y$ to a reasonable size, we can consider large set of data, as input or output. 

The reader can imagine also that choosing a relevant set $Y$ is a major source of optimization. We use this idea intensively in several applications. For instance, kernel clustering methods described in the section \@ref(a-kernel-based-clustering-algorithm) aims minimizing the error committed by our learning machine with respect to the set $Y = \mathcal{P}_{k}(X,Z)$, called **sharp discrepancy sequences** and defined below. We refer to this step as **learning process**, as this step is exactly the counterpart of the weight set for the neural network approach. This construction amounts to define a feed-backward machine, analogous to \@ref(eq:P), as
$$
f_z := \mathcal{P}_{k}(X,\mathcal{P}_{k}(X,Z),Z)f(X).
$$


Observe that \@ref(eq:P) allows us also to compute the following operation, where $\nabla := (\partial_1,\ldots,\partial_D)$ holds for the gradient
$$ 
 (\nabla f)(Z) := ( \nabla \mathcal{P}_{k} )(X,Y,Z)f(X):= (\nabla_z k)(Z,Y)K(X,Y)^{-1}f(X) \in \mathbb{R}^{D \times N_z \times D_f} (\#eq:dfm)
$$
meaning that $\nabla \mathcal{P}_{k} \in \mathbb{R}^{D \times N_z \times N_x}$ is interpreted as a tensor operator. This operator is described in Section \@ref(discrete-differential-operators), as well as numerous others, as for instance Laplacian, Leray, integral operators, that are based on it. They will be used in designing computational methods for problems involving partial differential equations (PDEs) and **differential learning machine** methods.


### Transportation maps

We will use surjective maps $S:\mathbb{R}^T \mapsto \mathbb{R}^D$, referred to as *transportation maps*, and we can distinguish between several types: 

- **rescaling maps** when $T=D$, in order properly the data $X,Y,Z$ to a given kernel,
- **dimension reduction maps** when $T \leq D$, or 
- **dimension augmentation** when $T \geq D$, when adding information to the training set is required. This transformation is sometimes called a **kernel trick**.

The list of maps available in our framework is given in the following table.

| | Maps | S(X)   |
|---|---|---|
|1. | Affine | $S(X)=$   |
|2.  | Exponential   | $S(X) = e^X$  |
|3.    | Identity | $S(X) = X$  |
|4.   | Log   | $S(X)$ = $\log(X)$  |
|5.   | Map to grid  |  $S(X)$ |
|6.   | Scale to standard deviation  |  $S(X) = \frac{x}{\sigma}$, $\sigma = \sqrt{\frac{1}{N_x}\sum_{n<N_x} (x^n - \mu)}$, $\mu = \frac{1}{N_x}\sum_{n<N_x} x^n$ |
|7.   | Scale to erf  | $S(X) = erf(x)$, $erf$ is the standard error function.  |
|8.   |  Scale to erfinv |  $S(X)=erf^{-1}(x)$,  $erf^{-1}$ is the inverse of $erf$. |
|9.   | Scale to mean distance  | $S(X) = \frac{x}{\sqrt{\alpha}}, \qquad \alpha = \sum_{i,k \le N_x} \frac{|x^i-x^k|^2}{N_x^2}.$|
|10.   | Scale to min distance  |  $S(X) = \frac{x}{\sqrt{\alpha}}, \qquad \alpha = \frac{1}{N_x} \sum_{i\le N_x} \min_{k \neq i}  |x^i-x^k|^2.$ |
|11.   | Scale to unit cube   | $S(X) = \frac{x - \min_n{x^n} + \frac{0.5}{N_x}}{\alpha}, \quad \alpha := \max_n{x^n} - \min_n{x^n}.$ |

Using a map $S$ amounts to change the kernel as $k(x,y) \mapsto k(S(x),S(y))$. For instance, for Gaussian kernels the Scale to min distance is usually a good choice: this map scales all points to the average min distance. Let us transform our Gaussian kernel with this map. Observe that, from the signature of the Gaussian setter function, we see that the Gaussian kernel is handled with the default map $set\_min\_distance\_map$. We do not discuss all optional parameters now, but refer the reader to a later section. 


In [ ]:
map_setters.set_standard_min_map()



$$
\begin{aligned}
kernel\_setters.set\_gaussian\_kernel(& polynomial\_order:int = 0,\\
& regularization:float = 1e-8, \\
& set\_map = map\_setters.set\_min\_distance\_map)
\end{aligned}
$$


### Extrapolation, interpolation, and projection

The Python function in our framework that describes the projection operator $\mathcal{P}_{k}$ is based on the definition in \@ref(eq:P), namely 
$$
  f_z = \text{op.projection}(X,Y,Z, f(X)=[], k=None, rescale = False) \in \mathbb{R}^{N_z \times D_f}. (\#eq:opP)
$$
The optional values in this function are as follows:

- The function $f(X)$ is optional, meaning that the user can retrieve the whole matrix $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_x}$ if desired.
- The kernel $k$ is optional, meaning that we let to the user the freedom to re-use an already input kernel. 
- The optional value \textit{rescale}, defaulted to false, allow to call the map prior of performing the projection operation \@ref(eq:P), in order to compute its internal states for proper data scaling. For instance, a rescaling computes $\alpha$ according to the set ($X,Y,Z$). 

Interpolation and extrapolation Python functions are, in agreement with \@ref(eq:EI), simple decorators toward the operator $\mathcal{P}_{k}$; see \@ref(eq:opP). Obviously, the main question arising at this stage is how good the approximation is $f_z$ compared to $f(Z)$, which is the question addressed in the next section. 
$$
 f_z = \text{op.extrapolation}(X,Z,f(X) = [],\ldots), \quad f_z = \text{op.interpolation}(X,Z,f(X) = [],\ldots) (\#eq:EIP)
$$

### Functional spaces and Kolmogorov decomposition

Given any finite collection of points $X = [x^1 ... x^{N_x}]$, $x^i\in \mathbb{R}^{D}, i= 1,...,N_x$, we introduce a (finite dimensional) vector space $\mathcal{H}_k^x$ consisting of all linear combinations of the \textit{basis functions} $x \mapsto k(x, x^n)$. In other words, we set
\begin{equation}
\mathcal{H}_k^x = \Big\{\sum_{1 \leq m \leq N_x} a_m k(\cdot, x^m) \,  / \,  a = (a^1, \ldots, a^{N_x}) \in \mathbb{R}^{N_x}  \Big\}. (\#eq:HKx)
\end{equation} 
The \textbf{functional space} $\mathcal{H}_{k}$ is (after suitably passing to a limit in \@ref(eq:HKx)) 
\begin{equation}
\mathcal{H}_k
=  \text{Span} \big\{k(\cdot, x) \, / \, x \in \mathbb{R}^D \big\}.(\#eq:Hk)
\end{equation} 
This space consists of all linear combinations of the functions $k(x, \cdot)$ (that is, parametrized by $x \in \mathbb{R}^D$) and is endowed with the scalar product 
\begin{equation}
\big\langle k(\cdot, x), k(\cdot, y) \big\rangle_{\mathcal{H}_{k}} = k(x,y), \qquad x, y \in \mathbb{R}^D. 
\end{equation}
On every finite dimensional subspace $\mathcal{H}_{k}^x \subset \mathcal{H}_{k}$ we can check that, according to the expression of the scalar product, 
\begin{equation}
\big\langle k(\cdot, x^i), k(\cdot, x^j) \big\rangle_{\mathcal{H}_{k}^x} = k(x^i,x) K(X,X)^{-1} k(x, x^j) = k(x^i,x^j), \ i,j = 1,...,N_x.
\end{equation}

The norm of any function $f$, in view of the functional space $\mathcal{H}_{k}$, is fully determined by the kernel $k$. A reasonable approximation of this norm, which is induced by the kernel matrix $K$ is given by 
$$
  \|f\|_{\mathcal{H}_{k}}^2 \sim f(X)^T K(X,X)^{-1} f(X)
$$
It is computed via the function in Python:
$$
    op.norm(X,Y,Z,f(X), set\_codpy\_kernel = None, rescale = True).
$$
Again we let the reader the choice to perform a rescaling of the kernel based on a transport map.


### Error estimates based on the generalized maximum mean discrepancy

Recall the notation for the projection operator \@ref(eq:P). Then the following estimation error holds for any vector-valued function $f:\mathbb{R}^D \mapsto \mathbb{R}^{D_f}$:
$$
   \Big| \frac{1}{N_x}\sum_{n=1}^{N_x} f(x^n) - \frac{1}{N_z}\sum_{n=1}^{N_z} f_{z^n} \Big| \le \Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big) \|f\|_{\mathcal{H}_{k}}.
$$
Before describing this formula, let us precise that it is a computationally tractable one, that can be systematically applied to check the pertinence of any kernel machine. It is also a quite general one: this formula can be adapted to others kind of error measuring. We have also 
$$
  \Big\| f(Z) - f_z \Big\|_{\ell^2(N_z)^{D_f}}\le \Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big) \|f\|_{\mathcal{H}_{k}}. (\#eq:err)
$$
This error formula can be split into two parts.


The first part, $\Big( d_k\big(X,Y\big) + d_k\big(Y,Z\big) \Big)$ measures some kernel-related distance between a set of points, that we call the \textbf{discrepancy functional}, known as the maximum mean discrepancy (MMD), first introduced in \cite{GR:2006}. It is a quite natural quantity, as one expects that the quality of an extrapolation degrades if the extrapolation set $Z$ move away from the sampling set $X$. Its definition is
$$
d_k\big(X,Y\big)^2 := \frac{1}{N_x^2}\sum_{n=1,m=1}^{N_x,N_x} k\big(x^n,x^m\big) + \frac{1}{N_y^2}\sum_{n=1,m=1}^{N_y,N_y} k\big(y^n,y^m\big) - \frac{2}{N_x N_y}\sum_{n=1,m=1}^{N_x,N_y} k\big(x^n,y^m\big) (\#eq:dk)
$$
and is described in the Python function
$$
  op.discrepancy(X,Y,Z,set\_codpy\_kernel = None, rescale = True)
$$
In particular, the user should pay attention to an undesirable rescaling effect due to the variable *rescale*. Section \@ref(a-study-of-the-discrepancy-functional) contains plots and some analysis of this functional. In this book we call generalized MMD and discrepancy error equivalently.

## Dealing with kernels 

### Maps and kernels 

**Maps can ruin your prediction**. We now compare the ground truth values $(Z,f(Z)) \in \mathbb{R}^{N_z \times D} \times \mathbb{R}^{N_z \times D_f}$ and the predicted values  $(Z,f_z)  \in \mathbb{R}^{N_z \times D} \times \mathbb{R}^{N_z \times D_f}$ . In Figure \@ref(fig:787).  we set a different map, called mean distance map, which scales all points to the average distance for a Gaussian kernel. This example illustrates how maps can drastically influence the computation. However, the very same map can be appropriate for other kernels; see Figure \@ref(fig:787).


In [ ]:
fz_extrapolated = op.extrapolation(x,fx,z,set_codpy_kernel = 
kernel_setters.set_gaussian_kernel(0,1e-8,map_setters.set_mean_distance_map),rescale = True)
matern_kernel = kernel_setters.set_matern_tensor_kernel(0,1e-8,map_setters.set_mean_distance_map)
fz_extrapolated1 = op.extrapolation(x,fx,z,set_codpy_kernel =matern_kernel,rescale = True)


In [ ]:
kwargs = {"mp_max_items" :4, "mp_ncols" : 3 }
multi_plot([(z,fz),(z,fz_extrapolated)] + [(z,fz_extrapolated1)],plot_trisurf,projection='3d', **kwargs)


**Composition of maps**. Maps can be composed together. For instance, consider the following Python definition of one of our maps, which we may use as a Swiss-knife map for Gaussian kernels: 



In [ ]:
def set_standard_min_map(**kwargs):
    map_setters.set_min_distance_map(**kwargs)
    pipe_map_setters.pipe_erfinv_map(**kwargs)
    pipe_map_setters.pipe_unitcube_map(**kwargs)


$$
\begin{aligned}
map\_setters.set\_min\_distance\_map(^{**}kwargs) \\
pipe\_map\_setters.pipe\_erfinv\_map() \\
    pipe\_map\_setters.pipe\_unitcube\_map()
\end{aligned}
$$    

For any $X \in \mathbb{R}^{N_x \times D}$, this composite map performs the following operations: first, rescale all data to the unit cube using scale to unit cube map; second, apply the map defined as $S(X) = erf^{-1}(2X - 1)$ and finally use the average min distance map.

### Illustration of different kernels predictions

As it is clear from previous sections, the external parameters of a kernel-based prediction machine are 

* In most situations, a kernel is defined by
  
    * a positive definite kernel function, 
    * a map.
    
* The choice of the inner parameters set $Y$. We usually face three main choices here.

    * $Y=X$, that corresponds to the *extrapolation* case; cf. Section \@ref(extrapolation-interpolation-and-projection). This is the most efficient choice when one seeks for high accuracy results.
    * $Y$ is randomly picked among $X$. This choice trades accuracy for execution time and is adapted to heavy training set.
    * $Y$ is set as a sharp discrepancy sequence of $X$, described in Section \@ref(a-kernel-based-clustering-algorithm). This choice optimizes accuracy versus execution time. These are the most possible accurate machine at a given computational burden but involves a time-consuming algorithm.

In order to illustrate the effects of different kernels and maps on learning machines, we compare predictions for the one-dimensional test described for different kernels.


In [ ]:
fun_extrakernel()



## Discrete differential operators 

### Coefficient operator
We start this section by further analyzing the projection operator \@ref(eq:P). We can interpret this operator in a basis function setting:
\begin{equation} 
 f_Z := K(Z,Y) c_Y, \quad c_Y:= K(X,Y)^{-1}f(X) \in \mathbb{R}^{N_Y \times D_f}. (\#eq:coefy)
\end{equation}
The coefficients $c_Y$ corresponds to the representation of $f$ in a basis of functions 
\begin{equation} 
 f_Z := \sum_{n=1}^{N_Y} c_y^n K(Z,y^n), 
\end{equation}
Coefficients are matrices also, having size $N_Y \times D_f$, except for some composite kernels. The table below shows the first four coefficients of the test function $f_z$.

### Partition of unity

For any $Y \in \mathbb{R}^{N_y \times D}$, consider the projection operator \@ref(eq:P), and the following vector-valued function: 
\begin{equation} 
 \phi : Y \mapsto \Big(\phi^1(Y),\ldots,\phi^{N_x}(Y) \Big) = K(Y,X)K(X,X)^{-1} \in \mathbb{R}^{N_y \times N_x}.(\#eq:PU) 
\end{equation}
that corresponds to the projection operator $\mathcal{P}_k(X,X,Y)$. On every point $x^n$, one computes (without considering regularization terms)
\begin{equation} 
 \phi(x^n):= \Big(0,\ldots,1,\ldots,0\Big) = \delta_{n,m}, 
\end{equation}
where $\delta_{n,m} = 1$ if $n=m$, $0$ else. For this reason, we call $\phi(x)$ a partition of unity. Figure \@ref(fig:part) illustrates partitions functions. 


In [ ]:
fun_part()



### Gradient operator

For any positive-definite kernel $k$, and points $X, Y, Z$, we define $\nabla$ operator as the 3-tensor: 
$$
  \nabla_{k}(X,Y,Z) = \Big(\nabla_z k\Big)(Z,Y)K(X,Y)^{-1} \in \mathbb{R}^{D \times N_x\times N_z}, (\#eq:nabla) 
$$
where $\Big(\nabla_z k\Big)(Z,Y) \in \mathbb{R}^{D \times N_x\times N_y}$ is interpreted as a three-tensor. The gradient of any vector valued function $f$, is computed as 
$$( \nabla f)(Z) \sim (\nabla_{k})(Z,Y,Z) f(X) \in \mathbb{R}^{D \times N_z\times D_f},$$

where we omit the dependence $\nabla_{k}(X,Y,Z)$ for concision. Observe that maps, introduced in Section \@ref(transportation-maps), modify the operator $\nabla_{k}$ as follows: 
\begin{equation}
 \nabla_{k\circ S} (X, Y, Z)  := (\nabla S)(Z)\Big(\nabla_1 k)(S(Z),S(Y) \Big) K(S(X),S(Y))^{-1}, (\#eq:nablak)
\end{equation}
where $\Big(\nabla_1 k\Big)(Z,Y) \in \mathbb{R}^{D \times N_z\times N_y}$ is interpreted as a three-tensor, as is $(\nabla S)(Z) := (\partial_d S^j)(Z^{n_z}) \in \mathbb{R}^{D \times D \times N_z}$, representing the Jacobian of the map $S$, and the multiplication holds for the two first indices.

**Two-dimensional example**. Figure \@ref(fig:nabla1) illustrate a corresponding derivative and compare it to the original one for the first and second dimensions respectively.


In [ ]:
kwargs = {"mp_max_items" :4, "mp_ncols" : 4 }
fun_nabla1()


### Divergence operator

We define the $\nabla^T$ operator as the transpose of the 3-tensor operator $\nabla$: 
$$
  <\nabla_{k}(X,Y,Z)f(X),g(Z)> = <f(X),\nabla_{k}(X,Y,Z)^T g(Z)>. 
$$
Hence, as the left-hand side is homogeneous with, for any smooth function $f$ and vector fields $g$, and $\nabla \cdot$ denotes the divergence operator
\begin{equation}
  \int (\nabla f) \cdot g d\mu = -\int f \nabla \cdot (g d\mu). (\#eq:578)
\end{equation}
The operator $\nabla^T$ is thus consistent with the divergence operator
$$
  \nabla_{k}(X,Y,Z)^T f(Z) \sim - \nabla \cdot (f\mu)(x)
$$
To compute this operator, we proceed as follows: starting from the definition of the gradient operator \@ref(eq:nablak), we define, for any $f(X) \in \mathbb{R}^{N_x \times D_f}$,$g(Z) \in \mathbb{R}^{D \times N_z \times D_f}$
$$
  <\Big(\nabla_z K\Big)(Z,Y)K(X,Y)^{-1}f_x,g_z> = <f_x,K(X,Y)^{-T}\Big(\nabla_z K\Big)(Z,Y)^T g_z>.
$$
Thus the operator $\nabla_{k}(X,Y,Z)$ is defined by 
$$
  \nabla_{k}(X,Y,Z)^T = K(X,Y)^{-T} \Big(\nabla_z K\Big)(Z,Y)^T \in \mathbb{R}^{N_x \times N_z D}, 
$$
where $\nabla_z K(Z,Y)^T \in \mathbb{R}^{N_y \times (N_z D)}$ is the transpose of the matrix $\nabla_z K(Z,Y)$. 

**A two-dimensional example**. Figure \@ref(fig:nablaTnabla) compares the outer product of the gradient to Laplace operator $\nabla_{k}(X,Y,Z)^T \nabla_{k}(X,Y,Z) f(X)$ to $\Delta_{k}(X,Y) f(X)$; see the next section. 


In [ ]:
fun_nablaTnabla()



### Laplace operator 

We define the Laplace operator as the matrix
$$
  \Delta_{k}(X,Y) = \Big(\nabla_{k}(X,Y,X)^{T}\Big)\Big(\nabla_{k}(X,Y,X)\Big) \in \mathbb{R}^{ N_x \times N_x}.
$$
This operator is not consistent with the "true" Laplace operator, but is instead consistent with \@ref(eq:578). 
$$
  -\nabla \cdot (\nabla f\mu).
$$

### Inverse Laplace operator

This operator is simply the pseudo-inverse of the Laplacian $\Delta_{k}(X,Y) \in \mathbb{R}^{N_x \times N_x}$.

**A two-dimensional example**. Figure \@ref(fig:Delta1) compares $f(X)$ with $\Delta_{k}(X,Y)^{-1}\Delta_{k}(X,Y)f(X)$. This latter operator is a projection operator (hence is stable).


In [ ]:
fun_Delta1()



We also compute the operator $\Delta_{k, x, y, z}\Delta_{k, x, y, z}^{-1}f(X)$ in Figure \@ref(fig:Delta2), to check that pseudo-inversion commutes, as highlighted by the following computations: 



In [ ]:
fun_Delta2()



### Integral operator - inverse gradient operator

The following operator $\nabla^{-1}_k$ is an integral-type operator
$$
  \nabla^{-1}_{k} =  \Delta_{k}^{-1}\nabla_{k}^T \in \mathbb{R}^{ N_x \times DN_z}. 
$$
It can be interpreted as a matrix, computed first considering $\nabla_{k}(X,Y,Z)\in \mathbb{R}^{D \times N_z\times N_x}$, down casting it to a matrix $\mathbb{R}^{ DN_z \times N_x}$ before performing a least-square inversion. This operator acts on any 3-tensor $v_z \in \mathbb{R}^{ D \times N_z \times D_{v_z}}$, and outputs a matrix
$$
  \nabla^{-1}_{k}(X, Y, Z) v_z \in \mathbb{R}^{ N_x \times D_{v_z}}, \quad v_z \in \mathbb{R}^{ D \times N_z \times D_{v_z}}
$$
The operator $\nabla^{-1}_{k}$ corresponds to the minimization procedure: 
$$
  h(X) := \arg \inf_{h \in \mathbb{R}^{ N_x \times D_{v_z}}}\| \nabla_{k}(X,Y,Z) h - v_z \|_{D,N_z,N_x}^2.
$$

**A two-dimensional example**. In Figure \@ref(fig:NablainvNabla) we show that $(\nabla )(\nabla )^{-1}f(X)$ coincides with $f(X)$. Observe however that this latter operation is not equivalent to Figure \@ref(fig:NablainvNabla2), which uses $Z$ as extrapolation set. 


In [ ]:
fun_NablainvNabla1()



In [ ]:
fun_NablainvNabla2()



### Integral operator - inverse divergence operator

The following operator $(\nabla_k^T)^{-1}$ is another integral-type operator of interest.
We define the $(\nabla^T)^{-1}$ operator as the pseudo-inverse of the $\nabla^T$ operator:
$$
  (\nabla^T_{k}(X,Y,Z))^{-1}= \nabla_{k}(X,Y,Z)\Delta_{k}(X,Y,Z)^{-1}.
$$

**A two-dimensional example**. We compute $\nabla_{k}(X,Y,Z)^T(\nabla^T_{k}(X,Y,Z))^{-1}= \Delta_{k}(X,Y,Z)\Delta_{k}(X,Y,Z)^{-1}$. Thus, the following computations should give comparable results as those obtained in the section concerning the inverse Laplace operator; see Section \@ref(inverse-laplace-operator).


In [ ]:
fun_Integral()



### Leray-orthogonal operator

We define the Leray-orthogonal operator as 
$$
  L_{k}(X,Y)^\perp := \nabla_{k}(X,Y) \Delta_{k}(X,Y)^{-1} \nabla_{k,x,y,x}^T = \nabla_{k}(X,Y,Z) \nabla_{k}(X,Y,Z)^{-1}. 
$$
This operator acts on any vector field $f(Z) \in \mathbb{R}^{ D \times N_z \times D_f}$, down casting it, performing a matrix multiplication and producing a three-tensor: 
$$
  L_{k}(X,Y,Z)^\perp f(Z) \in \mathbb{R}^{ D \times N_z \times D_f}.
$$
In Figure \@ref(fig:LerayT), we compare this operator to the original function $(\nabla f)(Z)$: 


In [ ]:
#LerayT_fz = op.Leray_T(z,y,nabla_fz)
LerayT()


### Leray operator and Helmholtz-Hodge decomposition

We define the Leray operator as 
$$
  L_{k}(X,Y,Z) := I_d-L_{k}(X,Y,Z)^\perp = I_d - \nabla_{k}(X,Y,Z) \Delta_{k}(X,Y,Z)^{-1} \nabla_{k}(X,Y,Z)^T, 
$$
where $I_d$ is the identity. Hence, we get the following orthogonal decomposition of any tensor fields:
$$
  v_z = L_{k}(X,Y,Z) v_z + L_{k}(X,Y,Z)^\perp v_z, \quad <L_{k}(X,Y,Z) v_z,L_{k}(X,Y,Z)^\perp v_z>_{D,N_z,D_v} = 0.
$$
This agrees with the Helmholtz-Hodge decomposition, decomposing any vector field into an orthogonal sum of a gradient and a divergence free vector:
$$
  v = \nabla h + \zeta, \quad \nabla \cdot \zeta = 0, \quad h := \Delta^{-1} \nabla \cdot v 
$$
Here we have also an orthogonal decomposition from a numerical point of view:
$$
  v_z = \nabla_{k}(X,Y,Z) h_x + \zeta_z, \quad h_x := \nabla_{k}(X,Y,Z)^{-1} v_z, \quad \zeta_z := L_{k}(X,Y,Z) v_z, 
$$
satisfying numerically
$$
  \nabla_{k}(X,Y,Z)^T \zeta_z = 0, \quad \left<\zeta_z, \nabla_{k}(X,Y,Z) h_x\right>_{D \times N_z \times D_f} = 0.
$$
In Figure \@ref(fig:Leray) we compare this operator to the original function $(\nabla f)(Z)$.


In [ ]:
Leray()



## Kernel engineering

### Manipulating kernels

In this section we describe some general operations on kernels, which allow us to generate new and relevant kernels. These operations preserve a positiveness property required for kernels. For this section, two kernels denoted by $k_i(x,y) : \mathbb{R}^D \times \mathbb{R}^D \mapsto \mathbb{R}, i=1,2$ are given with corresponding matrices $K_1$ and $K_2$. In agreement with \@ref(eq:P), we introduce the corresponding two projection operators:
\begin{equation} 
  \mathcal{P}_{k_i}(X,Y,Z):= K_i(Z,Y)K_i(X,Y)^{-1} \in \mathbb{R}^{N_z \times N_x}, i = 1,2 (\#eq:Pi)
\end{equation}

In order to work with two (or more) kernels, we introduced the following Python function, which are basic *setters* and *getters* to kernels: *get\_kernel\_ptr()* and *set\_kernel\_ptr(kernel\_ptr)*. The first one allows us to recover an already input kernel of our library, while the second one allows us to input it into our framework.

### Adding kernels

The first operation, denoted by \(k_1 + k_2\) and defined from any two kernels, consists in simply adding two kernels. If $K_1$ and $K_2$ are two kernel matrices associated to the kernels $k_1$ and $k_2$, then we define the sum of two kernels as $K(X,Y)\in \mathbb{R}^{N_x \times N_y}$ and corresponding projection operator as $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_y}$:
\begin{equation}
K(X,Y):=K_1(X,Y)+K_2(X,Y), \quad \mathcal{P}_{k}(X,Y,Z) = K(Z, X)K(X,Y)^{-1}. (\#eq:add)
\end{equation}

The functional space generated by \(k_1+k_2\) is then  
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k_1(\cdot, x^m)+k_2(\cdot, x^m))\Big\}. 
\end{equation}

### Multiplicating kernels

A second operation, denoted by $k_1 \cdot k_2$ and defined from any two kernels, consists in multiplying two kernels together. A kernel matrix $K(X,Y)\in \mathbb{R}^{N_x \times N_y}$ and a projection operator $\mathcal{P}_{k}(X,Y,Z) \in \mathbb{R}^{N_z \times N_y}$ corresponding to the product of two kernels are defined as
\begin{equation}
K(X,Y):=K_1(X,Y) \circ K_2(X,Y), \quad \mathcal{P}_{k}(X,Y,Z) = K(Z,X)K(X,Y)^{-1},  (\#eq:mul)
\end{equation}
where $\circ$ is the Hadamard product of two matrices. The functional space generated by \(k_1 \cdot k_2\) is
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k_1(\cdot, x^m)k_2(\cdot, x^m))\Big\}.
\end{equation}

### Convoluting kernels

Our next operation, denoted by \(k_1 \ast k_2\) and defined from any two kernels, consists in multiplying corresponding kernel matrices $K_1$ and $K_2$ as 
\begin{equation}
K(X,Y):=K_1(X,Y) K_2(Y,Y), (\#eq:ast)
\end{equation}
where $K_1(X,Y) K_2(Y,Y)$ stands for the standard matrix multiplication. The projection operator is given by $\mathcal{P}_{k}(X,Y,Z) = K(Z,X)K(X,Y)^{-1}$. Suppose that $k_1(x,y) = \varphi_1(x-y)$, $k_2(x,y) = \varphi_2(x-y)$, then
the functional space generated by \(k_1 \ast k_2\) is 
\begin{equation}
\mathcal{H}_k = \Big\{\sum_{1 \leq m \leq N_x} a^m (k(\cdot, x^m))\Big\}, 
\end{equation}
where $k(x,y) := \Big( \varphi_1 \ast \varphi_2 \Big)(x-y)$ is the convolution of the two kernels.

### Piped kernels

Another important operation is referred to here as "piping kernels" and provides yet another route for generating new kernels in a natural and explicit way.  It is denoted by \(k_1 | k_2\) and corresponds to define the projection operator \@ref(eq:opP) as follows: 
\begin{equation}
  \mathcal{P}_{k}(X,Y,Z) = \mathcal{P}_{k_1}(X,Y,Z) \pi_1(X,Y) + \mathcal{P}_{k_2}(X,Y,Z)\Big( I_d-\pi_1(X,Y)\Big), (\#eq:pipe)
\end{equation} 
where the projection operator here reads 
\[
 \pi_1(X,Y):=K_1(X,Y)K_1(X,Y)^{-1} = \mathcal{P}_{k_1}(X, Y, X).
\]
This operation splits the projection operator $\mathcal{P}_{k}(X,Y,Z)$ into two parts. The first part is handled by a single kernel, while the second kernel handles the remaining error. From a mathematical standpoint point, this is equivalent to a functional Gram-Schmidt orthogonalization process of both functional spaces $\mathcal{H}_{k_1}^x$, $\mathcal{H}_{k_2}^x$, and the corresponding functional space defined by \@ref(eq:pipe) is, after \@ref(eq:Hk), 
\begin{equation}
\mathcal{H}_k^x = \Big\{\sum_{1 \leq m \leq N_x} a^m k_1(\cdot, x^m) + \sum_{1 \leq m \leq N_x} b^m k_2(\cdot, x^m)  \Big\}. 
\end{equation}
Hence, this doubles up the coefficients \@ref(eq:coefy). We define its inverse concatenating matrix 
\begin{equation}
  K^{-1}(X,Y) = \Big( K_1(X,Y)^{-1} , K_2(X,Y)^{-1}\big( I_{N_x}-\pi_1(X,Y)\big)\Big) \in \mathbb{R}^{2N_y \times N_x}.
\end{equation}
The kernel matrix associated to a "piped" kernel pair is then
\begin{equation}
  K(X,Y) = \Big(K_1(X,Y) , K_2(X,Y) \Big) \in \mathbb{R}^{N_x \times 2 N_y}. 
\end{equation}


### Piping scalar product kernels: an example with a polynomial regression

Let $S : \mathbb{R}^D \mapsto \mathbb{R}^N$ be given by a family of $N$ basis functions $\varphi_n$, i.e. $S(x) = \Big(\varphi_1(x),\ldots,\varphi_N(x)\Big)$ and consider the following kernel, called dot product kernel (which is conditionally positive-definite): 
\begin{equation}
	k_1(x,y):= <S(x),S(y)>. (\#eq:LRK)
\end{equation}
Now, given any positive kernel $k_2(x,y)$, consider a "pipe" kernel $k_1 | k_2$. In particular, such a construction is very useful with a polynomial basis function $S(x) = \big(1,x_1,\ldots\big)$ : it consists of a classical polynomial regression, allowing to perfectly match moments of distributions, since the remaining error is handled by the second kernel.

### Neural networks viewed as kernel methods

Our setup describes the strategies developed in deep learning theory, which are based on neural networks. Namely, we consider any feed-forward neural network of depth $M$, defined by  
$$
z_m = y_{m} g_{m-1}(z_{m-1})\in \mathbb{R}^{N_{m} }, \qquad y_m \in \mathbb{R}^{N_{m} \times N_{m-1}}, 
\qquad z_0 = y_0 \in \mathbb{R}^{N_0}, 
$$
in which $y_0,\ldots,y_M$ are considered as weights and $g_{m}$ as prescribed activation functions. By concatenation, we arrive at the function 
$$
z_M(y) = y_M z_{M-1}(y_0, \ldots,y_{M-1}) : \mathbb{R}^{N_0 \times \ldots \times N_M} \mapsto \mathbb{R}^{N_M }.
$$
This neural network is thus entirely represented by the kernel composition
$$
k(y_m,\ldots,y_0) = k_m \Big(y_m, k_{m-1} \big( \ldots, k_1(y_1,y_0) \big) \Big) \in \mathbb{R}^{ N_m \times \ldots \times N_0},
$$ 
where $k_m(x,y) = g_{m-1}(x y^T)$, indeed $z_M(y) = y_M k(y_{M-1},\ldots,y_0)$. 

## A first application: a clustering algorithm

### Distance-based unsupervised learning machines 

In this section we describe a kernel-based clustering algorithm. This algorithm, already presented with a toy example in Section \@ref(clustering), is also benchmarked in a forthcoming section devoted to more concrete problems; see Chapter \@ref(application-for-unsupervised-machine-learning).

Distance-based minimization algorithms can be thought as finding a minimum of a distance between set of points $d(X, Y)$, defining equivalently a distance between discrete measures $\mu_x$, $\mu_y$. Within this setting, minimization problem can be expressed as
$$
Y = \arg \inf_{Y \in \mathbb{R}^{N_y \times D}} d(X,Y)  (\#eq:dist).
$$


Suppose that this last problem is well-posed, assuming that the distance functional is convex\footnote{although most of existing distance are not convex}. Once the cluster set $Y:=\Big(y^1,\ldots,y^{N_y} \big)$ is computed, then one can define the index function $\sigma(w, Y) := \arg \inf_{j=1 \ldots N_Y} \{ d(w, y^j)\}$, as for \@ref(eq:sigmaw). One we can extend naturally this function, defining a map
\begin{equation}
\sigma(Z,Y) := \{\sigma(z^1,Y), \ldots, \sigma(z^{N_z},Y)\} \in [1,\ldots,N_y]^{N_z}, (\#eq:sigma)
\end{equation}
that acts on the indices of the test set $Z$. This allows to compare this prediction to a given, user-desired, partition of $f(Z)$, if needed. 

Note that the function $\sigma(Z, Y)$ is surjective (many-to-one). Hence we can define its injective (one-to-many) inverse, $\sigma(Z, Y)^{-1}(n)$, describing those points of the test set attached to one $y^n$. This construction defines cells, very similarly to quantization, $C^n := \sigma(\mathbb{R}^D, y^n)^{-1}(n)$, defining a partition of unity of the space $\mathbb{R}^D$.
A last remark: consider, in the context of supervised clustering methods, the training set and its values $X, f(X)$ and the index map $\sigma(X, Y) \in [1, \ldots, N_x]^{N_y}$ defined above. One can always define a prediction on the test set $Z$ as
$$
f_z := f\big(X^{\sigma(Y^{\sigma(Z, Y)},X )}\big), (\#eq:pm)
$$
showing that a distance-minimization unsupervised algorithm naturally defines a supervised one.

### Sharp discrepancy sequences 

Our kernel-based algorithm for clustering can be described as follows:

* The unsupervised algorithm aims to solve the minimization problem \@ref(eq:dist) with the MMD or the discrepancy functional \@ref(eq:dk). 
This procedure is separated into two main steps.
  * First solve a discrete version of \@ref(eq:dist), namely
$$
X^\sigma = \arg \inf_{\sigma \in \Sigma } d_k(X, X^\sigma), 
$$
where $\Sigma$ denotes the set of all subsets from $[1,\ldots,N_y] \mapsto [1,\ldots,N_x]$, and $d_k$ is the discrepancy functional. This minimization problem is described Chapter \@ref(general-cost-functions-and-motivations).
  * Depending on kernels, this step is completed by a simple gradient descent algorithm. The initial state for this minimization is chosen to be $X^\sigma$.
  * The resulting solution $Y$ is named **sharp discrepancy sequences**.
* The supervised algorithm consists then simply to compute the projection operator \@ref(eq:P), that we recall here.
$$
 f_z := \mathcal{P}_{k}(X,Y,Z)f(X)
$$
using the python function \@ref(eq:opP), where the *weight set* $Y$ is taken as the sharp discrepancy sequence computed above.

### Python functions

* The unsupervised clustering algorithm is given by the Python function
$$
  sharp\_discrepancy(X, Y = [], N_y =0, set\_codpy\_kernel = None, rescale = False,nmax=10).
$$
* Let $X \in \mathbb{R}^{N_x\times D}$, $Y \in \mathbb{R}^{N_y\times D}$ any two distributions of points and $k(x,y)$ a positive-definite kernel. The following Python function
$$
  codpy.alg.match(Y,X,nmax = 10)
$$
provides an approximation to the following problem
$$
  \arg \inf_{Y \in \mathbb{R}^{N_y\times D}} d_k(X,Y)^2
$$
via a simple descent algorithm: starting from the input distribution $Y$, the algorithm performs $nmax$ steps of a descent algorithm and output the resulting distribution.
* The computation of index associations \@ref(eq:sigma), that is the function $\sigma_{d_k}(X, Y)$, is given by
$$
alg.distance\_labelling(X, Y, set\_codpy\_kernel = None, rescale = True).
$$
This function relies on the distance matrix $D(X,Y)$; see \@ref(fundamental-notions-for-supervised-learning).

### Impact of sharp discrepancy sequences on discrepancy errors

Figure \@ref(clustering) presented a first illustration of the impact of computing discrepancy errors on several toy "blob" examples. In this paragraph, we fix the number of "blobs" to two, and the number of generated points $N_x$ to 100. We then follow the test methodology of Section \@ref(clustering), re-running all tests with scenarios for $N_y$ covering [0,100]. Figure \@ref(fig:741) compare the results for discrepancy errors of the three methods. One can check visually that discrepancy errors is zero, whatever the clustering method is, when the number of clusters $N_y$ tends to $N_x$. Note also that our kernel clustering methods shows quite good inertia performance indicators. This is surprising, as our method is based on discrepancy error minimization, not inertia. An interpretation could be that the inertia functional is bounded by the discrepancy error one.


In [ ]:
fun_741()



### A study of the discrepancy functional

As stated in the previous section, we first compute a discrete minimizing problem and denote $X^\sigma$ its solution. We eventually complete this step with a simple gradient descent algorithm. This section explains and motivate this choice. Indeed, the minimizing properties $d_k(X, Y)$ relies heavily on the kernel definition $k(x, y)$, and we face an alternative, depending on regularity of kernels, that we illustrate numerically in this section:

* If the kernel is smooth, then the distance functional $d_k(X, Y)$ also is, and a descent algorithm based on gradients computations is an efficient option.
* If the kernel is only continuous, or piecewise derivable, then we assume that the minimum is attained by the discrete minimum solution $X^\sigma$.

Hence, we study in this section the effect of some classical kernel over this functional for a better understanding.
To that aim, let us produce some random distributions $x \in \mathbb{R}^{N_x}$ in one dimension, we will study then for three kernels the following functionals:
$$
  \mathbb{R}^{N_y} \ni y \mapsto d_k(x, y), 
$$
$$
  \mathbb{R}^{N_y \times 2} \ni Y=(y^1, y^2) \mapsto d_k(x,Y).
$$

We generate uniform random variables $x \in \mathbb{R}^{N_x},y\in \mathbb{R}^{N_y}$ and $Y = (y^1, y^2) \in \mathbb{R}^{N_y \times 2}$.


In [ ]:
D,Nx,Ny,Nz=2,1000,500,2
x,y,z,Nx,Ny,Nz =  data_random_generator().get_raw_data(D=1,Nx=5,Ny=1000,Nz=0)
(y2,dummy,dummy,Nx2,dummy,dummy) = data_random_generator().get_raw_data(D=2,Nx=1000,Ny=0,Nz=0)
def compare_plot_list(xfx, ax = None, fun_x = get_data, extra_plot_fun = None, **kwargs):
    listxs = xfx[0:2]
    listfxs = xfx[2:4]
    from matplotlib.dates import date2num
    if ax: return compare_plot_lists_ax(listxs, listfxs, ax, fun_axvspan = None, **kwargs)
    index = kwargs.get("index",0)
    labelx=kwargs.get("labelx",'x-units')
    labely=kwargs.get("labely",'f(x)-units')
    # listlabels=kwargs.get("listlabels",["curve:"+str(n) for n in range(len(listxs))])
    listlabels=kwargs.get("listlabels",[None for n in range(len(listxs))])
    listalphas=kwargs.get("alphas",np.repeat(1.,len(listxs)))
    xscale =kwargs.get("xscale",None)
    yscale =kwargs.get("yscale",None)
    figsize =kwargs.get("figsize",(2,2))
    Show = kwargs.get("Show",True)
    plt.figure(figsize=figsize)
    for x,fx,label,alpha in zip(listxs, listfxs,listlabels,listalphas):
        plotx = fun_x(x)
        plotfx = get_data(fx)
        plotx,plotfx,permutation = lexicographical_permutation(plotx,plotfx,**kwargs)
        if extra_plot_fun is not None: extra_plot_fun(plotx,plotfx)
        # if fun_axvspan is not None: plt.axvspan(xmin=date2num(plotx[int(len(plotx)*.8)]),xmax= date2num(plotx[-1]), color='y', alpha=0.5, lw=0)
        # plotx = np.array(x)[permutation]
        plt.plot(plotx,plotfx,marker = 'o',ls='-',label= label, markersize=12 / len(plotx),alpha = alpha)
        plt.legend()
    # plt.xscale(xscale)
    # plt.yscale(yscale)
    title = kwargs.get("title",'')
    plt.title(title)
    plt.xlabel(labelx)
    plt.ylabel(labely)
    if Show: plt.show()


**An example of smooth kernels: Gaussian**. We start our study of the discrepancy functional with a Gaussian kernel. The Gaussian kernels is a family of kernels based upon the following kernel, generating functional spaces of smooth functions.
$$
k(x,y) = \exp(-(x-y)^2)
$$
The following picture shows the function $y \mapsto d_k(x,y)$ in blue. We also display the function $d_k(x,x^n)$, $n=1 \ldots N_x$ in Figure \@ref(fig:MMD1) in orderto illustrate that this functional is neither convex nor concave.


In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = kernel_setters.set_gaussian_kernel)
dysg = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxsg = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
dzsg = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))


We see that the functional $y \mapsto d_k(x,y)$ admits a minimum which is close to $y=\frac{1}{2}$, as expected. Figure \@ref(fig:MMD3) displays $y:=(y^1,y^2) \mapsto d_k(x,y)$. We see that this functional admits two minima, and this reflects the fact that the functional $y \mapsto d_k(x,y)$ is invariant by permutation of the indices of $y$.

**An example of Lipschitz continuous kernels: RELU**. Let us now consider a kernel which generate a functional space with less regularity. RELU kernels is the following family of kernels (essentially generating the space of functions with bounded variation):
$$
k(x, y) = 1 -|x-y|. 
$$


In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = alg.set_sampler_kernel)
dysn = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxsn = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
dzsn = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))


As is clear in Figure \@ref(fig:MMD1) the function $y \mapsto d_k(x,y)$ is only piecewise differentiable. Hence in some cases, the functional $d_k(x,y)$ might have an infinity of solutions (here on the "flat" segment). Figure \@ref(fig:MMD3) displays $y:=(y^1,y^2) \mapsto d_k(x,y)$  for a two-dimensional example

**An example of continuous kernel: Matern**. The Matern kernel reads (and generates a space of continuous functions)
$$
k(x,y) = \exp(-|x-y|).
$$


In [ ]:
DF = op.discrepancy_functional(x=x,y=z,set_codpy_kernel = kernel_setters.set_matern_tensor_kernel)
dysm = DF.eval(y.reshape(y.shape[0],1,y.shape[1]))
dxsm = DF.eval(x.reshape(x.shape[0],1,x.shape[1]))
dzsm = DF.eval(y2.reshape(y2.shape[0],2,int(y2.shape[1] / 2)))


In Figure \@ref(fig:MMD1) we see that the function $y \mapsto d_k(x,y)$ admits concave parts, and a gradient-descent algorithm can not give satisfactory results. Figure \@ref(fig:MMD3) shows a two-dimensional example $y:=(y^1,y^2) \mapsto d_k(x,y)$ 



In [ ]:
kwargs = {"mp_max_items" :4, "mp_ncols" : 4 }
list = [(y,x,dysg,dxsg),(y,x,dysn,dxsn),(y,x,dysm,dxsm)]
title_list = ["Gaussian kernel", "RELU kernel", "Matern kernel"]
multi_plot(list,compare_plot_list, f_names = title_list, **kwargs)


In [ ]:
kwargs = {"mp_max_items" :4, "mp_ncols" : 4 }
title_list = ["Gaussian kernel", "RELU kernel", "Matern kernel"]
multi_plot([(y2,dzsg),(y2,dzsn),(y2,dzsm)],plot_trisurf,projection='3d',elev=30, f_names = title_list,**kwargs)


## Bibliography
There esxists a vast literature on RKHS methods (cf. the list of refetences at the end of this monograph) and, in particular on kernel regressions. A good reference is the textbook "Elements of Statistical Learning Data Mining, Inference, and Prediction" by R.Tibshirani et al. RKHS methods in statistics and related fields are described in the textbook "Reproducing Kernel Hilbert Spaces in Probability and Statistics" by C. Thomas-Agnan et al. This also studied in "A Hilbert Space Embedding for Distributions" by A. Smola, A. Gretton et al. A dimension reduction technique for kernel least-square models was introduced in "Kernel Partial Least Squares Regression in Reproducing Kernel Hilbert Space" by R. Rosipal and L. Trejo. 

## Appendix to Chapter 3

### Maps and kernels

In the table below we propose a list of kernels with maps that best fit to each kernel

| | Kernels | Maps   |
|---|---|---|
|1. | Dot Product |  |
|2.  | RELU   |  |
|3.    | Gaussian  | Affine map, mean distance map,   |
|4.   | Periodic Gaussian   | Affine map  |
|5.   | Matern norm  |   |
|6.   | Matern tensor  |   |
|7.   | Matern periodic  |   |
|8.   |  Multiquadric norm |   |
|9.   | Multiquadric tensor  |   |
|10.   | Sincard square tensor  |   |
|11.   | sincard tensor   |   |
|12.   | Tensor norm  | Unit cube map  |
|13.   |  Truncated norm |   |
|14.   |  Truncated periodic |   |
